In [0]:
import tensorflow as tf
import numpy as np
import keras
import pandas as pd
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
x_test = pd.read_csv('/content/drive/My Drive/Machine Learning/github/practice/MNIST/test.csv')
train = pd.read_csv('/content/drive/My Drive/Machine Learning/github/practice/MNIST/train.csv')
print(train.shape)
x_test.shape


(42000, 785)


(28000, 784)

In [0]:
train.head()
x_train = train.drop('label', axis=1)
y_train = train['label']
print(x_train.shape, y_train.shape)
x_train = np.array(x_train)
x_test = np.array(x_test)

(42000, 784) (42000,)


In [0]:
# from sklearn.model_selection import train_test_split

# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

In [0]:
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
# x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_val = x_val.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
# x_val /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])
# print('Number of images in x_val', x_val.shape[0])

x_train shape: (42000, 28, 28, 1)
Number of images in x_train 42000
Number of images in x_test 28000


In [0]:
# x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)


In [0]:
from keras.utils.np_utils import to_categorical

In [0]:
y_train = to_categorical(y_train, num_classes = 10)
# y_val = to_categorical(y_val, num_classes=10)

In [0]:
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LeakyReLU, BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping,LearningRateScheduler
from keras.models import load_model
from keras.optimizers import RMSprop


In [0]:
#Callbacks
checkpoint = ModelCheckpoint('/content/model.h5',save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=7, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
callbacks = [checkpoint, learning_rate_reduction]

In [0]:
nets = 15
model = [0] *nets
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
batch_size = 128
nb_epoch = 100
from sklearn.model_selection import train_test_split
epochs=30
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu', input_shape = (28,28,1)))
    model[j].add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu'))
    model[j].add(MaxPooling2D(pool_size=(2,2)))
    model[j].add(Dropout(rate=0.5))


    model[j].add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model[j].add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model[j].add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    model[j].add(Dropout(rate=0.5))


    model[j].add(Flatten())
    model[j].add(Dense(256, activation = "relu"))
    model[j].add(Dropout(rate=0.5))
    model[j].add(Dense(10, activation = "softmax"))
    model[j].compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
    






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model

In [0]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
early = EarlyStopping(patience=10)
# TRAIN NETWORKS
history = [0] * nets
epochs = 50
for j in range(nets):
    # X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.1)
    history[j] = model[j].fit(x_train,y_train, batch_size=64,
        epochs = epochs,   
        validation_split = 0.2, callbacks=callbacks, verbose=1, shuffle = True)
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 33600 samples, validate on 8400 samples
Epoch 1/50





33600/33600 [==============================] - 9s 267us/step - loss: 0.3175 - acc: 0.8946 - val_loss: 0.0700 - val_acc: 0.9775
Epoch 2/50
33600/33600 [==============================] - 5s 150us/step - loss: 0.1072 - acc: 0.9681 - val_loss: 0.0465 - val_acc: 0.9848
Epoch 3/50
33600/33600 [==============================] - 5s 149us/step - loss: 0.0826 - acc: 0.9745 - val_loss: 0.0654 - val_acc: 0.9811
Epoch 4/50
33600/33600 [==============================] - 5s 143us/step - loss: 0.0709 - acc: 0.9783 - val_loss: 0.0396 - val_acc: 0.9882
Epoch 5/50
33600/33600 [==============================] - 5s 146us/step - loss: 0.0630 - acc: 0.9813 - val_loss: 0.0348 - val_acc: 0.9883
Epoch 6/50
33600/33600 [==============================] - 5s 148us/step - loss: 0.0553 - acc: 0.9830 - val_loss: 0.0306 - val_acc: 0.9918
Epoch 7/50
33600/3360

In [0]:
# # model = load_model('/content/model.h5')
# results = model.predict(x_test)

# results = np.argmax(results,axis = 1)
# results = pd.Series(results,name="Label")
# submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

# submission.to_csv("MNIST_submission-5.csv",index=False)


In [0]:
# ENSEMBLE PREDICTIONS AND SUBMIT
results = np.zeros( (x_test.shape[0],10) ) 
for j in range(nets):
    results = results + model[j].predict(x_test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("MNIST-CNN-ENSEMBLE-1.csv",index=False)

In [0]:
submission


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
